# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff6b45a23d0>
├── 'a' --> tensor([[ 1.4264, -0.3906, -0.7825],
│                   [-0.4324,  1.2545,  1.2197]])
└── 'x' --> <FastTreeValue 0x7ff6b45a2670>
    └── 'c' --> tensor([[-0.3817,  0.4550, -0.1760, -0.6889],
                        [ 0.3615,  0.1771,  0.7097, -0.6870],
                        [-0.0628,  0.2529,  1.5790,  1.9756]])

In [4]:
t.a

tensor([[ 1.4264, -0.3906, -0.7825],
        [-0.4324,  1.2545,  1.2197]])

In [5]:
%timeit t.a

59.7 ns ± 0.0833 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff6b45a23d0>
├── 'a' --> tensor([[ 2.1754,  0.7958,  0.2325],
│                   [ 1.7422, -0.8706,  1.4803]])
└── 'x' --> <FastTreeValue 0x7ff6b45a2670>
    └── 'c' --> tensor([[-0.3817,  0.4550, -0.1760, -0.6889],
                        [ 0.3615,  0.1771,  0.7097, -0.6870],
                        [-0.0628,  0.2529,  1.5790,  1.9756]])

In [7]:
%timeit t.a = new_value

57.6 ns ± 0.0315 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.4264, -0.3906, -0.7825],
               [-0.4324,  1.2545,  1.2197]]),
    x: Batch(
           c: tensor([[-0.3817,  0.4550, -0.1760, -0.6889],
                      [ 0.3615,  0.1771,  0.7097, -0.6870],
                      [-0.0628,  0.2529,  1.5790,  1.9756]]),
       ),
)

In [10]:
b.a

tensor([[ 1.4264, -0.3906, -0.7825],
        [-0.4324,  1.2545,  1.2197]])

In [11]:
%timeit b.a

58.5 ns ± 0.0541 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1469, -0.1717, -0.3282],
               [-0.9413, -1.1595, -1.4400]]),
    x: Batch(
           c: tensor([[-0.3817,  0.4550, -0.1760, -0.6889],
                      [ 0.3615,  0.1771,  0.7097, -0.6870],
                      [-0.0628,  0.2529,  1.5790,  1.9756]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.144 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

837 ns ± 2.12 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 30.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 616 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 452 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff60783bb80>
├── 'a' --> tensor([[[ 1.4264, -0.3906, -0.7825],
│                    [-0.4324,  1.2545,  1.2197]],
│           
│                   [[ 1.4264, -0.3906, -0.7825],
│                    [-0.4324,  1.2545,  1.2197]],
│           
│                   [[ 1.4264, -0.3906, -0.7825],
│                    [-0.4324,  1.2545,  1.2197]],
│           
│                   [[ 1.4264, -0.3906, -0.7825],
│                    [-0.4324,  1.2545,  1.2197]],
│           
│                   [[ 1.4264, -0.3906, -0.7825],
│                    [-0.4324,  1.2545,  1.2197]],
│           
│                   [[ 1.4264, -0.3906, -0.7825],
│                    [-0.4324,  1.2545,  1.2197]],
│           
│                   [[ 1.4264, -0.3906, -0.7825],
│                    [-0.4324,  1.2545,  1.2197]],
│           
│                   [[ 1.4264, -0.3906, -0.7825],
│                    [-0.4324,  1.2545,  1.2197]]])
└── 'x' --> <FastTreeValue 0x7ff688a39160>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 120 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff6b459ac40>
├── 'a' --> tensor([[ 1.4264, -0.3906, -0.7825],
│                   [-0.4324,  1.2545,  1.2197],
│                   [ 1.4264, -0.3906, -0.7825],
│                   [-0.4324,  1.2545,  1.2197],
│                   [ 1.4264, -0.3906, -0.7825],
│                   [-0.4324,  1.2545,  1.2197],
│                   [ 1.4264, -0.3906, -0.7825],
│                   [-0.4324,  1.2545,  1.2197],
│                   [ 1.4264, -0.3906, -0.7825],
│                   [-0.4324,  1.2545,  1.2197],
│                   [ 1.4264, -0.3906, -0.7825],
│                   [-0.4324,  1.2545,  1.2197],
│                   [ 1.4264, -0.3906, -0.7825],
│                   [-0.4324,  1.2545,  1.2197],
│                   [ 1.4264, -0.3906, -0.7825],
│                   [-0.4324,  1.2545,  1.2197]])
└── 'x' --> <FastTreeValue 0x7ff6b4595040>
    └── 'c' --> tensor([[-0.3817,  0.4550, -0.1760, -0.6889],
                        [ 0.3615,  0.1771,  0.7097, -0.6870],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 63.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.6 µs ± 157 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.4264, -0.3906, -0.7825],
                [-0.4324,  1.2545,  1.2197]],
       
               [[ 1.4264, -0.3906, -0.7825],
                [-0.4324,  1.2545,  1.2197]],
       
               [[ 1.4264, -0.3906, -0.7825],
                [-0.4324,  1.2545,  1.2197]],
       
               [[ 1.4264, -0.3906, -0.7825],
                [-0.4324,  1.2545,  1.2197]],
       
               [[ 1.4264, -0.3906, -0.7825],
                [-0.4324,  1.2545,  1.2197]],
       
               [[ 1.4264, -0.3906, -0.7825],
                [-0.4324,  1.2545,  1.2197]],
       
               [[ 1.4264, -0.3906, -0.7825],
                [-0.4324,  1.2545,  1.2197]],
       
               [[ 1.4264, -0.3906, -0.7825],
                [-0.4324,  1.2545,  1.2197]]]),
    x: Batch(
           c: tensor([[[-0.3817,  0.4550, -0.1760, -0.6889],
                       [ 0.3615,  0.1771,  0.7097, -0.6870],
                       [-0.0628,  0.2529,  1.5790,  1.9756]],
         

In [26]:
%timeit Batch.stack(batches)

79 µs ± 159 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.4264, -0.3906, -0.7825],
               [-0.4324,  1.2545,  1.2197],
               [ 1.4264, -0.3906, -0.7825],
               [-0.4324,  1.2545,  1.2197],
               [ 1.4264, -0.3906, -0.7825],
               [-0.4324,  1.2545,  1.2197],
               [ 1.4264, -0.3906, -0.7825],
               [-0.4324,  1.2545,  1.2197],
               [ 1.4264, -0.3906, -0.7825],
               [-0.4324,  1.2545,  1.2197],
               [ 1.4264, -0.3906, -0.7825],
               [-0.4324,  1.2545,  1.2197],
               [ 1.4264, -0.3906, -0.7825],
               [-0.4324,  1.2545,  1.2197],
               [ 1.4264, -0.3906, -0.7825],
               [-0.4324,  1.2545,  1.2197]]),
    x: Batch(
           c: tensor([[-0.3817,  0.4550, -0.1760, -0.6889],
                      [ 0.3615,  0.1771,  0.7097, -0.6870],
                      [-0.0628,  0.2529,  1.5790,  1.9756],
                      [-0.3817,  0.4550, -0.1760, -0.6889],
                      [ 0.3615,  

In [28]:
%timeit Batch.cat(batches)

146 µs ± 343 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

319 µs ± 1.56 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
